## Project Description

Imagine you're developing a deep learning system tailored for sentiment analysis of product reviews, specifically for a newly established online beautiy product retail company. The goal is to assist the company in making informed decisions about inventory management – deciding what products to retain and what to remove from stock. The company, keen on enhancing customer satisfaction, has been actively monitoring comments on their website and has invested in annotators to label sentiments. They hand you a dataset comprising 80,000 customer reviews, each labeled with 0 for negative sentiment and 1 for positive sentiment. After extensive effort and refinement, you successfully train and deploy a classifier that predicts sentiment based on online comments. Excitedly, you report an 86% accuracy on a held-out test set to your bosses. However, to your disappointment, management expresses dissatisfaction, insisting on a minimum of 90% accuracy before considering the widespread implementation of the AI model. 
You suspect that certain annotators might have made errors, potentially affecting your model's effectiveness. Empowered by a newfound "confidence," you opt for "confidence" learning to pinpoint and rectify any inaccuracies in the dataset before embarking on the retraining process once more.

First, we prepare the environment for AWS SageMaker operations by setting up clients and retrieving essential configuration details like the default S3 bucket, execution role, and AWS region. 

In [1]:
pip install -r requirements.txt

  Using cached aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (31 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 13.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 6.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 19.7 MB/s eta 0:00:0000:01
Using cached aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached aiosignal-1.3.1-py3-none-any.wh

In [1]:
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
import json
import botocore
from botocore.exceptions import ClientError

config = botocore.config.Config(user_agent_extra='dlai-pds/c2/w3')

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', 
                  config=config)

sm_runtime = boto3.client('sagemaker-runtime',
                          config=config)

sess = sagemaker.Session(sagemaker_client=sm,
                         sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


We then configure the data source for a training job in SageMaker, defining where the training data is located (in this case, an S3 bucket) and the nature of the data.

In [2]:
from sagemaker.inputs import TrainingInput

# TODO: set the path to the train data
train_data = TrainingInput(
    s3_data='s3://sagemaker-us-east-1-403320517955/data/',
    content_type='application/x-sagemaker-training-data'
)

validation_data = TrainingInput(
    s3_data='s3://sagemaker-us-east-1-403320517955/data/',
    content_type='application/x-sagemaker-training-data'
)


## Testing the access: 

In [3]:
import boto3

# Initialize a session using your credentials
session = boto3.Session(
    aws_access_key_id='YOUR_AWS_ACCESS_KEY_ID',
    aws_secret_access_key='YOUR_AWS_SECRET_ACCESS_KEY',
    region_name='us-east-1'
)

# Create an S3 client
s3 = session.client('s3')

# List objects within a specified bucket and prefix
response = s3.list_objects_v2(Bucket='sagemaker-us-east-1-403320517955', Prefix='data/')

# Print the contents of the directory
for item in response.get('Contents', []):
    print(item['Key'])


data/
data/dev.csv
data/dev.pt
data/test.csv
data/test.pt
data/train.csv
data/train.pt


## The estimator:

A PyTorch estimator with the specified configurations for a SageMaker training job is created. The training job will use the provided entry point script, run on the specified instance type, and output the trained model to the specified S3 path. The entry point script main.py contains the main steps that needs to be completed in this project.

In [4]:
from sagemaker.pytorch import PyTorch

# Define the S3 bucket and prefix
bucket = 'sagemaker-us-east-1-403320517955'
prefix = 'models'


# TODO: create the estimator
estimator = PyTorch(
    entry_point= "main.py",
    source_dir= "./",
    base_job_name="sagemaker-script-mode",
    role=role,
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    framework_version="2.1",
    py_version="py310",
    dependencies= ['src', 'requirements.txt'],
    output_data_config={
        'S3OutputPath': f's3://{bucket}/{prefix}/output'  # Defined S3 path for output data
    },
    output_path=f's3://{bucket}/{prefix}/output',  # Define S3 path for output data
    environment={'PYTHONPATH': 'src'}
)


The following script sets up a ModelCheckpoint callback to automatically save the best model (based on development loss) during the training process in a SageMaker training job. The best model will be stored at the specified directory path within the SageMaker environment.

In [5]:
# Save the best model during training by specifying the output path
# (Note: The output path should be where the best model will be saved within the S3 bucket)
model_checkpoint = {
    'ModelCheckpoint': {
        'monitor': 'avg_val_loss',
        'dirpath': '/opt/ml/model/',
        'filename': 'best_model',
        'save_top_k': 1,
        'mode': 'min'
    }
}

# Attach the ModelCheckpoint callback to the estimator
estimator._hyperparameters['callbacks'] = [model_checkpoint]

Starting the training process: 

In [6]:
# TODO: train the model
estimator.fit({'train': train_data, 'validation': validation_data})


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: sagemaker-script-mode-2024-05-19-22-49-53-926


2024-05-19 22:49:57 Starting - Starting the training job...
2024-05-19 22:49:57 Pending - Training job waiting for capacity......
2024-05-19 22:51:05 Pending - Preparing the instances for training...
2024-05-19 22:51:58 Downloading - Downloading the training image..................
2024-05-19 22:54:59 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-05-19 22:55:22,196 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-05-19 22:55:22,217 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-05-19 22:55:22,230 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-05-19 22:55:22,232 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-05-19 22:55:24,478 sagemaker-training-toolkit INFO     Installing de

## Model Deployment

We need to copy the training artifacts, i.e, output.tar.gz, from the corresponding S3 bucket to the current working directory.

In [11]:
#TODO: copy the training artifacts from the S3 bucket to the current working directory

bucket_name = "sagemaker-us-east-1-403320517955"
s3_output_path = "s3://{}/models/output/sagemaker-script-mode-2024-05-19-22-49-53-926/output".format(bucket_name)

!aws s3 cp {s3_output_path} ./ --recursive  

download: s3://sagemaker-us-east-1-403320517955/models/output/sagemaker-script-mode-2024-05-19-22-49-53-926/output/model.tar.gz to ./model.tar.gz
download: s3://sagemaker-us-east-1-403320517955/models/output/sagemaker-script-mode-2024-05-19-22-49-53-926/output/output.tar.gz to ./output.tar.gz


We can decompress the training artifacts to `extracted_files` for further exploration.

In [12]:
!tar -xzf output.tar.gz -C extracted_training_artifacts


tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'


We then create an endpoint 'sentiment-analysis-endpoint-2' and deploy the model to that endpoint.

In [13]:
# TODO: deploy the trained model
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="sentiment-analysis-endpoint-2"
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-403320517955/models/output/sagemaker-script-mode-2024-05-19-22-49-53-926/output/model.tar.gz), script artifact (s3://sagemaker-us-east-1-403320517955/sagemaker-script-mode-2024-05-19-22-49-53-926/source/sourcedir.tar.gz), and dependencies (['src', 'requirements.txt']) into single tar.gz file located at s3://sagemaker-us-east-1-403320517955/sagemaker-script-mode-2024-05-19-23-06-01-033/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: sagemaker-script-mode-2024-05-19-23-06-01-033
INFO:sagemaker:Creating endpoint-config with name sentiment-analysis-endpoint-2
INFO:sagemaker:Creating endpoint with name sentiment-analysis-endpoint-2


------!

In [14]:
# Delete the endpoint to prevent any potential costs
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: sentiment-analysis-endpoint-2
INFO:sagemaker:Deleting endpoint with name: sentiment-analysis-endpoint-2
